In [ ]:
"""
Load libs
"""
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from utils.openrouter import get_openrouter_responses
import yaml
from datasets import load_dataset
import re
import json
import re

seed = 123

load_dotenv('.env')
no_sample_params = {'temperature': 0, 'top_p': 1, 'topk_k': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'repetition_penalty': 1}

## Prepare Raw Data

In [ ]:
"""
Load dataset
"""
def load_harm_df():
    strong_reject_ds = load_dataset('walledai/StrongREJECT', split = 'train').to_list()
    strong_reject_df =\
        pd.DataFrame(strong_reject_ds)\
        .rename(columns = {'prompt': 'harm_question', 'category': 'harm_category'})\
        .assign(harm_source = 'StrongREJECT')

    return pd.concat([strong_reject_df])\
        .assign(harm_index = lambda df: list(range(0, len(df))))\
        .reset_index(drop = True)\
        [['harm_index', 'harm_question', 'harm_category', 'harm_source']]

harm_df = load_harm_df()
harm_df

## Generate Synthetic Policies

In [ ]:
"""
Synthetic policy generation
"""
def get_policy_creation_prompts(harm_questions):
    base_prompt = yaml.safe_load(open('./prompts/create-policies.yaml', 'r'))
    input_prompts = [
        base_prompt + [{'role': 'user', 'content': f"`<TEST_PROMPT>{x}</TEST_PROMPT>`"}]
        for x in harm_questions
    ]
    return input_prompts

synthetic_policy_df = harm_df.assign(synthetic_policy_prompts = get_policy_creation_prompts(harm_df['harm_question'].tolist()))
display(synthetic_policy_df)
synthetic_policy_df['synthetic_policy_prompts'].tolist()[0]

In [ ]:
"""
Generate synthetic policies
"""
def _validate_and_extract_response(llm_response, key = 'content'):
    """
    Extract content/reasoning from response
    
    Params:
        @llm_response: The LLM response object
        @key: The key to extract, either 'content' or 'reasoning'
    """
    if 'choices' not in llm_response:
        print(llm_response)
        return {'reasoning': None, 'output': None}

    choice = llm_response['choices'][0]
    if choice['finish_reason'] == 'length':
        print(f"Warning - early stop: {choice['finish_reason']}")
        print(f"  CONTENT: {choice['message']['content']}")
        print(f"  REASONING: {choice['message']['reasoning']}")
        return {'reasoning': None, 'output': None}

    return {
        'reasoning': choice['message']['reasoning'],
        'output': choice['message']['content'],
    }

def _extract_synthetic_policy(s):
    if s is None:
        return None
    m = re.search(r'(?s)<SYNTHETIC_POLICY>(.*?)</SYNTHETIC_POLICY>', s)
    return m.group(1) if m else None

async def get_synthetic_policies(synthetic_policy_df, model, model_provider):
    
    raw_llm_responses = await get_openrouter_responses(
        synthetic_policy_df['synthetic_policy_prompts'].tolist(),
        {
            'model': model,
            'provider': {'order': [model_provider], 'allow_fallbacks': False},
            'max_tokens': 20_000,
            **no_sample_params
        },
        batch_size = 20
    )
    
    extracted_llm_responses = [_validate_and_extract_response(x) for x in raw_llm_responses]
    
    policy_df_for_model =\
        pd.DataFrame({
            'synthetic_policy': [_extract_synthetic_policy(x['output']) for x in extracted_llm_responses],
            'harm_index': synthetic_policy_df['harm_index'].tolist(),
            'synthetic_policy_model': model
        })

    return policy_df_for_model

synthetic_policy_df = await get_synthetic_policies(synthetic_policy_df, 'google/gemini-2.5-pro', 'google-ai-studio')
synthetic_policy_df

In [ ]:
"""
Flag missing/refused synthetic prompts & write CSV
"""
synthetic_policy_export_df =\
    harm_df\
    .merge(synthetic_policy_df, on = 'harm_index', how = 'left')\
    .assign(
        has_policy = lambda df: np.where((df['synthetic_policy'].notna()) & (df['synthetic_policy'].str.len() >= 20), 1, 0),
    )

print(f"Has policy: {np.sum(synthetic_policy_export_df['has_policy'].tolist())}/{len(synthetic_policy_export_df)}")
synthetic_policy_export_df.to_csv('data-and-policies.csv', index = False)

## Create Destyled Policies

In [ ]:
"""
Import if running below section separately
"""
synthetic_policy_export_df =\
    pd.read_csv('./data-and-policies.csv')\
    .pipe(lambda df: df[df['has_policy'] == 1])

In [ ]:
"""
Generate syntactically normal variant
Creates same prompt as before but with a extra assistant prompt (the previous generation) as well as 
"""

destyle_prompt =\
"""
Please rewrite your previous synthetic policy in more normal language. You should still keep the content the same, but stylistically reword it to avoid such unusual syntactic style, repetition, and terminology. Keep it within one paragraph still. Respond with the <SYNTHETIC_POLICY></SYNTHETIC_POLICY> tags as before.
""".strip()

def get_destyled_policy_prompt(inputs_and_policies):
    base_prompt = yaml.safe_load(open('./prompts/create-policies.yaml', 'r'))
    input_prompts = [
        base_prompt +\
            [{'role': 'user', 'content': f"`<TEST_PROMPT>{row['harm_question']}</TEST_PROMPT>`"}] +\
            [{'role': 'assistant', 'content': f"<SYNTHETIC_POLICY>{row['synthetic_policy']}<SYNTHETIC_POLICY>"}] +\
            [{'role': 'user', 'content': destyle_prompt}]
        for row in inputs_and_policies
    ]
    return input_prompts

destyled_policy_df = synthetic_policy_export_df.assign(destyled_policy_prompts = get_destyled_policy_prompt(synthetic_policy_export_df.to_dict('records')))
display(destyled_policy_df)
destyled_policy_df['destyled_policy_prompts'].tolist()[0]

In [ ]:
"""
Generate destyled policies
"""
def _validate_and_extract_response(llm_response, key = 'content'):
    """
    Extract content/reasoning from response
    
    Params:
        @llm_response: The LLM response object
        @key: The key to extract, either 'content' or 'reasoning'
    """
    if 'choices' not in llm_response:
        print(llm_response)
        return {'reasoning': None, 'output': None}

    choice = llm_response['choices'][0]
    if choice['finish_reason'] == 'length':
        print(f"Warning - early stop: {choice['finish_reason']}")
        print(f"  CONTENT: {choice['message']['content']}")
        print(f"  REASONING: {choice['message']['reasoning']}")
        return {'reasoning': None, 'output': None}

    return {
        'reasoning': choice['message']['reasoning'],
        'output': choice['message']['content'],
    }

def _extract_synthetic_policy(s):
    if s is None:
        return None
    m = re.search(r'(?s)<SYNTHETIC_POLICY>(.*?)</SYNTHETIC_POLICY>', s)
    return m.group(1) if m else None

async def get_destyled_policies(destyled_policy_df, model, model_provider):
    
    raw_llm_responses = await get_openrouter_responses(
        destyled_policy_df['destyled_policy_prompts'].tolist(),
        {
            'model': model,
            'provider': {'order': [model_provider], 'allow_fallbacks': False},
            'max_tokens': 20_000,
            **no_sample_params
        },
        batch_size = 20
    )
    
    extracted_llm_responses = [_validate_and_extract_response(x) for x in raw_llm_responses]
    
    policy_df_for_model =\
        pd.DataFrame({
            'destyled_policy': [_extract_synthetic_policy(x['output']) for x in extracted_llm_responses],
            'harm_index': destyled_policy_df['harm_index'].tolist(),
            'destyled_policy_model': model
        })

    return policy_df_for_model

destyled_policy_df = await get_destyled_policies(destyled_policy_df, 'google/gemini-2.5-pro', 'google-ai-studio')
destyled_policy_df

In [ ]:
"""
Flag missing/refused destyled prompts & write CSV - this writes the destyled policies only. Merge this with `data-and-policies.csv` for later use.
"""
destyled_policy_export_df =\
    destyled_policy_df\
    .assign(
        has_destyled_policy = lambda df: np.where((df['destyled_policy'].notna()) & (df['destyled_policy'].str.len() >= 20), 1, 0),
    )

print(f"Has policy: {np.sum(destyled_policy_export_df['has_destyled_policy'].tolist())}/{len(destyled_policy_export_df)}")
destyled_policy_export_df.to_csv('destyled-policies.csv', index = False)